In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#CNN Regression model for solar irradiance nowcasting from IR sky images.
class SolarCNNRegression(nn.Module):


    def __init__(self, input_channels=3, num_classes=1):
        super(SolarCNNRegression, self).__init__()

        # Feature extraction layers (Convolutional backbone)
        self.features = nn.Sequential(
            # First conv block
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 240x320 -> 120x160

            # Second conv block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 120x160 -> 60x80

            # Third conv block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 60x80 -> 30x40

            # Fourth conv block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 30x40 -> 15x20

            # Fifth conv block
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling
        )

        # Regression head for irradiance prediction
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)  # Output: solar irradiance value
        )

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Extract features
        features = self.features(x)

        # Regression prediction
        irradiance = self.regressor(features)

        return irradiance

#Extract features for LSTM input
    def get_features(self, x):

        with torch.no_grad():
            features = self.features(x)
            return features.flatten(1)  # Flatten for sequence input


class SolarCNNWithFeatureExtraction(SolarCNNRegression):


    def __init__(self, input_channels=3, feature_dim=512):
        super().__init__(input_channels, 1)
        self.feature_dim = feature_dim

        # Feature projection layer for LSTM input
        self.feature_projector = nn.Sequential(
            nn.Linear(512, feature_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

    def forward(self, x, return_features=False):
        # Extract convolutional features
        conv_features = self.features(x)
        flattened_features = conv_features.flatten(1)

        # Get irradiance prediction
        irradiance = self.regressor[-3:](
            self.regressor[:-3](flattened_features)
        )

        if return_features:
            # Project features for LSTM input
            projected_features = self.feature_projector(flattened_features)
            return irradiance, projected_features

        return irradiance


# Legacy model for backward compatibility
class SolarCNN(SolarCNNRegression):

    pass

LSTM model for solar irradiance forecasting


In [ ]:
import torch
import torch.nn as nn
from scripts.cnn_model import SolarCNNWithFeatureExtraction
class SolarLSTMForecasting(nn.Module):
    """
    LSTM model for solar irradiance forecasting following the paper methodology.
    Uses bidirectional LSTM with 2 layers and 128 hidden units per direction.
    """

    def __init__(self, input_size=1, hidden_size=128, num_layers=2, output_size=4, dropout=0.2):
        super(SolarLSTMForecasting, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        # Bidirectional LSTM with 2 layers as specified in paper
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Fully connected layers as specified in paper
        # Input size is hidden_size * 2 (bidirectional)
        lstm_output_size = hidden_size * 2

        self.fc_layers = nn.Sequential(
            nn.Linear(lstm_output_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(64, output_size)  # Predict next 4 timestamps
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
                # Set forget gate bias to 1
                n = param.size(0)
                start, end = n // 4, n // 2
                param.data[start:end].fill_(1.)

        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)

        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Take the output from the last time step
        # lstm_out shape: (batch_size, sequence_length, hidden_size * 2)
        last_output = lstm_out[:, -1, :]  # (batch_size, hidden_size * 2)

        # Pass through fully connected layers
        forecast = self.fc_layers(last_output)

        return forecast


class HybridCNNLSTM(nn.Module):
    """
    Hybrid CNN-LSTM model as described in the paper.
    CNN extracts features from images, LSTM performs temporal forecasting.
    """

    def __init__(self, cnn_model=None, feature_dim=512, sequence_length=20,
                 lstm_hidden_size=128, forecast_horizon=4):
        super(HybridCNNLSTM, self).__init__()

        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon

        # CNN component for nowcasting (pre-trained or trainable)
        if cnn_model is None:
            self.cnn = SolarCNNWithFeatureExtraction(feature_dim=feature_dim)
        else:
            self.cnn = cnn_model

        # LSTM component for forecasting
        self.lstm = SolarLSTMForecasting(
            input_size=1,  # Input is the nowcast irradiance values
            hidden_size=lstm_hidden_size,
            output_size=forecast_horizon
        )

        # Freeze CNN if using pre-trained model
        self.freeze_cnn = False

    def set_cnn_trainable(self, trainable=True):
        """Control whether CNN parameters are trainable"""
        for param in self.cnn.parameters():
            param.requires_grad = trainable
        self.freeze_cnn = not trainable

    def forward(self, image_sequence):
        """
        Forward pass for the hybrid model

        Args:
            image_sequence: Tensor of shape (batch_size, sequence_length, channels, height, width)

        Returns:
            nowcasts: Current irradiance predictions for each image
            forecasts: Future irradiance predictions
        """
        batch_size, seq_len, channels, height, width = image_sequence.shape

        # Reshape for CNN processing
        images_flat = image_sequence.view(-1, channels, height, width)

        # CNN nowcasting for each image in sequence
        nowcasts = self.cnn(images_flat)  # (batch_size * seq_len, 1)
        nowcasts = nowcasts.view(batch_size, seq_len, 1)  # (batch_size, seq_len, 1)

        # LSTM forecasting using nowcast sequence
        forecasts = self.lstm(nowcasts)  # (batch_size, forecast_horizon)

        return nowcasts, forecasts

    def predict_from_sequence(self, image_sequence):
        """Prediction method for inference"""
        self.eval()
        with torch.no_grad():
            nowcasts, forecasts = self.forward(image_sequence)
        return nowcasts, forecasts


# Simple LSTM for backward compatibility
class SolarLSTM(nn.Module):
    """Legacy LSTM model - kept for compatibility"""

    def __init__(self, input_size=1, hidden_size=128, output_size=1):
        super(SolarLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output





In [12]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import json

def load_multi_day_irradiance(files):
    """Load and concatenate irradiance data from multiple CSV files"""
    all_values = []
    for f in files:
        df = pd.read_csv(f)
        values = df.iloc[:, 1].values.astype(np.float32)
        all_values.extend(values)
    return np.array(all_values)

class MultiDayTimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, irradiance_files, sequence_length, forecast_horizon):
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon
        self.irradiance = load_multi_day_irradiance(irradiance_files)
        self.length = len(self.irradiance) - sequence_length - forecast_horizon + 1

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        seq = self.irradiance[idx:idx+self.sequence_length]
        target = self.irradiance[idx+self.sequence_length:idx+self.sequence_length+self.forecast_horizon]
        seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(-1)
        target = torch.tensor(target, dtype=torch.float32)
        return seq, target

class LSTMTrainer:
    """
    Trainer class for LSTM forecasting model following paper methodology
    """

    def __init__(self, config=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Default configuration based on paper
        self.config = {
            'sequence_length': 20,
            'forecast_horizon': 4,
            'learning_rate': 1e-4,
            'batch_size': 64,
            'num_epochs': 5,
            'lstm_hidden_size': 128,
            'lstm_num_layers': 2,
            'dropout': 0.2,
            'weight_decay': 1e-4,
            'scheduler_patience': 10,
            'early_stopping_patience': 15,
            'save_dir': 'models',
            'log_dir': 'logs'
        }

        if config:
            self.config.update(config)

        # Create directories
        os.makedirs(self.config['save_dir'], exist_ok=True)
        os.makedirs(self.config['log_dir'], exist_ok=True)

        # Initialize model
        self.model = SolarLSTMForecasting(
            input_size=1,
            hidden_size=self.config['lstm_hidden_size'],
            num_layers=self.config['lstm_num_layers'],
            output_size=self.config['forecast_horizon'],
            dropout=self.config['dropout']
        ).to(self.device)

        # Initialize optimizer and scheduler
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=self.config['learning_rate'],
            weight_decay=self.config['weight_decay']
        )

        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            patience=self.config['scheduler_patience'],
            factor=0.5
        )

        self.criterion = nn.MSELoss()

        # Training history
        self.train_losses = []
        self.val_losses = []
        self.best_val_loss = float('inf')
        self.patience_counter = 0

    def train_epoch(self, train_loader):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0.0
        num_batches = 0

        with tqdm(train_loader, desc="Training LSTM") as pbar:
            for batch_idx, (sequences, targets) in enumerate(pbar):
                sequences, targets = sequences.to(self.device), targets.to(self.device)

                self.optimizer.zero_grad()

                outputs = self.model(sequences)
                loss = self.criterion(outputs, targets)

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                self.optimizer.step()

                total_loss += loss.item()
                num_batches += 1

                pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

        return total_loss / num_batches

    def validate_epoch(self, val_loader):
        """Validate for one epoch"""
        self.model.eval()
        total_loss = 0.0
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            with tqdm(val_loader, desc="Validation") as pbar:
                for sequences, targets in pbar:
                    sequences, targets = sequences.to(self.device), targets.to(self.device)

                    outputs = self.model(sequences)
                    loss = self.criterion(outputs, targets)

                    total_loss += loss.item()

                    all_predictions.append(outputs.cpu().numpy())
                    all_targets.append(targets.cpu().numpy())

                    pbar.set_postfix({'Val Loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(val_loader)

        predictions = np.concatenate(all_predictions, axis=0)
        targets = np.concatenate(all_targets, axis=0)

        rmse_per_step = []
        mae_per_step = []

        for step in range(self.config['forecast_horizon']):
            step_pred = predictions[:, step]
            step_target = targets[:, step]
            mse = mean_squared_error(step_target, step_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(step_target, step_pred)
            rmse_per_step.append(rmse)
            mae_per_step.append(mae)

        overall_rmse = np.sqrt(mean_squared_error(targets.flatten(), predictions.flatten()))
        overall_mae = mean_absolute_error(targets.flatten(), predictions.flatten())

        return avg_loss, overall_rmse, overall_mae, rmse_per_step, mae_per_step

    def train(self, train_loader, val_loader=None):
        """Main training loop"""
        print(f"Starting LSTM training for {self.config['num_epochs']} epochs...")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"Sequence length: {self.config['sequence_length']}, Forecast horizon: {self.config['forecast_horizon']}")

        for epoch in range(self.config['num_epochs']):
            print(f"\nEpoch {epoch+1}/{self.config['num_epochs']}")

            train_loss = self.train_epoch(train_loader)
            self.train_losses.append(train_loss)
            print(f"Train Loss: {train_loss:.4f}")

            if val_loader is not None:
                val_loss, overall_rmse, overall_mae, rmse_per_step, mae_per_step = self.validate_epoch(val_loader)
                self.val_losses.append(val_loss)

                print(f"Val Loss: {val_loss:.4f}, Overall RMSE: {overall_rmse:.2f} W/m²")
                print(f"RMSE per step: {[f'{r:.2f}' for r in rmse_per_step]}")

                self.scheduler.step(val_loss)

                if val_loss < self.best_val_loss:
                    self.best_val_loss = val_loss
                    self.patience_counter = 0

                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'val_loss': val_loss,
                        'rmse': overall_rmse,
                        'config': self.config
                    }, os.path.join(self.config['save_dir'], 'best_lstm_model.pth'))

                    print(f"New best model saved! RMSE: {overall_rmse:.2f} W/m²")
                else:
                    self.patience_counter += 1

                if self.patience_counter >= self.config['early_stopping_patience']:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break

        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'train_losses': self.train_losses,
            'val_losses': self.val_losses
        }, os.path.join(self.config['save_dir'], 'final_lstm_model.pth'))

        history = {
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'config': self.config
        }

        with open(os.path.join(self.config['log_dir'], 'lstm_training_history.json'), 'w') as f:
            json.dump(history, f, indent=2)

        print("LSTM training completed!")
        return self.train_losses, self.val_losses

    def load_model(self, checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        return checkpoint

    def evaluate(self, test_loader):
        self.model.eval()
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            for sequences, targets in tqdm(test_loader, desc="Evaluating LSTM"):
                sequences, targets = sequences.to(self.device), targets.to(self.device)
                outputs = self.model(sequences)
                all_predictions.append(outputs.cpu().numpy())
                all_targets.append(targets.cpu().numpy())

        predictions = np.concatenate(all_predictions, axis=0)
        targets = np.concatenate(all_targets, axis=0)

        overall_rmse = np.sqrt(mean_squared_error(targets.flatten(), predictions.flatten()))
        overall_mae = mean_absolute_error(targets.flatten(), predictions.flatten())

        rmse_per_step = []
        for step in range(self.config['forecast_horizon']):
            step_rmse = np.sqrt(mean_squared_error(targets[:, step], predictions[:, step]))
            rmse_per_step.append(step_rmse)

        print(f"Test Results - Overall RMSE: {overall_rmse:.2f} W/m²")
        print(f"RMSE per forecast step: {[f'{r:.2f}' for r in rmse_per_step]}")

        return overall_rmse, overall_mae, rmse_per_step, predictions, targets

def train_lstm_forecasting():
    """Main function to train LSTM forecasting model"""

    # Configuration
    config = {
        'sequence_length': 20,
        'forecast_horizon': 4,
        'learning_rate': 1e-4,
        'batch_size': 64,
        'num_epochs': 50,
        'irradiance_files': [
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_15/pyranometer/2019_01_15.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_16/pyranometer/2019_01_16.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_17/pyranometer/2019_01_17.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_18/pyranometer/2019_01_18.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_19/pyranometer/2019_01_19.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_20/pyranometer/2019_01_20.csv'
            # Add more days as needed
        ]
    }

    # Create dataset
    print("Loading multi-day time series dataset...")
    dataset = MultiDayTimeSeriesDataset(
        irradiance_files=config['irradiance_files'],
        sequence_length=config['sequence_length'],
        forecast_horizon=config['forecast_horizon']
    )

    # Split dataset
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=2
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=2
    )

    print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

    trainer = LSTMTrainer(config=config)
    train_losses, val_losses = trainer.train(train_loader, val_loader)

    print("LSTM forecasting training completed!")

if __name__ == '__main__':
    train_lstm_forecasting()

Loading multi-day time series dataset...
Train samples: 435458, Val samples: 108865
Using device: cuda
Starting LSTM training for 50 epochs...
Model parameters: 199,684
Sequence length: 20, Forecast horizon: 4

Epoch 1/50


Training LSTM: 100%|██████████| 6805/6805 [00:46<00:00, 145.86it/s, Loss=19221.2031]


Train Loss: 140300.1966


Validation: 100%|██████████| 1702/1702 [00:09<00:00, 184.16it/s, Val Loss=235380.1094]


Val Loss: 112591.2913, Overall RMSE: 335.44 W/m²
RMSE per step: ['335.69', '335.09', '335.67', '335.31']
New best model saved! RMSE: 335.44 W/m²

Epoch 2/50


...

Val Loss: 1147.3094, Overall RMSE: 33.85 W/m²
RMSE per step: ['33.62', '33.73', '33.92', '34.15']

Epoch 50/50


Training LSTM: 100%|██████████| 6805/6805 [00:50<00:00, 134.75it/s, Loss=15.0220]


Train Loss: 1175.5708


Validation: 100%|██████████| 1702/1702 [00:09<00:00, 178.86it/s, Val Loss=3267.4819]


Val Loss: 1166.1147, Overall RMSE: 34.13 W/m²
RMSE per step: ['33.89', '34.01', '34.20', '34.43']
LSTM training completed!
LSTM forecasting training completed!
